# Custom sampling

You can define your own sampling procedure, using your own software for data calculation. Here you can find an example how to start adaptive sampling with your own functions

## Import libraries

In [ ]:
from pyfitit import *

## Define the input files generator

Its arguments: params dictionary and folder to run calculations inside it

In [ ]:
def moleculeConstructor(params):
    mol = Molecule('../Feterpy inverse method/Fe_terpy.xyz')
    mol.atom[1,1] -= params['shift1']
    mol.atom[30,1] += params['shift2']
    return mol

def generateInput(params, folder):
    fdmnes.generateInput(molecule=moleculeConstructor(params), energyRange='-15 0.1 8 0.5 18 1 30 4 54 6 117', 
                         Green=True, folder=folder)

## Define function to calculate folder

In [ ]:
def run(folder):
    exe = fdmnes.findFDMNES()
    output, returncode = utils.runCommand(exe, folder, outputTxtFile='output.txt')
    if returncode != 0:
        raise Exception('Error while executing "'+exe+'" command:\n'+output)
    return output

## Define results parser

It should return one Spectrum object or dict of spectra {name:spectrum}

In [ ]:
def parseOneFolder(folder):
    return parseFdmnesFolder(folder)

## Define create DataFrame function

Its arguments: allData dict {folder:result of parseOneFolder}, parentFolder, goodFolders - list of relative good folder names

The function should return pandas.DataFrame with column names 'e_???' where ??? - energy values corresponding to spectrum intensity values. Or dictionary of such DataFrames

If neccessary it can change list goodFolders, than it should return df_spectra, goodFolders, badFolders

In [ ]:
import numpy as np
import pandas as pd
def createDataframes(allData, parentFolder, goodFolders):
    # make common energy
    energyCount = np.array([len(allData[folder].x) for folder in goodFolders])
    n = np.max(energyCount)
    # change goodFolders array because some calculations could be broken
    goodFolders = [folder for folder in goodFolders if len(allData[folder].x) == n]
    allEnergies = np.array([allData[folder].x for folder in goodFolders])
    energies = np.median(allEnergies, axis=0)
    energies = np.sort(energies)
    spectra = np.zeros([len(goodFolders), energies.size])
    for i,d in enumerate(goodFolders):
        spectra[i, :] = np.interp(energies, allData[d].x, allData[d].y)
    df_spectra = pd.DataFrame(data=spectra, columns=['e_' + str(e) for e in energies])
    badFolders = list(set(allData.keys()) - set(goodFolders))
    return df_spectra, goodFolders, badFolders

## Define sample preprocessor

Is applied to y before given it to adaptive sampler, it is also applied before plotting. Is NOT applied before saving sample. It should work for sample and for one spectrum. 

func(sample)->sample (sample - Sample of output of parseAllFolders)

func(spectrum)->spectrum (spectrum - output of parseOneFolder) 

In [ ]:
expSpectrum = readSpectrum('../Feterpy inverse method/exp_ground.txt').limit(interval=[7100, 7200])

def samplePreprocessor(sample):
    smoothParams = {'Gamma_hole': 4.23, 'Ecent': 53, 'Elarg': 24, 'Gamma_max': 24, 'Efermi': 7115, 
                    'shift': 136, 'norm':0.0323}

    def spectrumPreprocessor(spectrum):
        smoothed = smoothInterpNorm(smoothParams, spectrum, smoothType='fdmnes', expSpectrum=expSpectrum)[0]
        return smoothed

    if isinstance(sample, ML.Sample):
        spectra = [spectrumPreprocessor(sample.getSpectrum(i)) for i in range(sample.getLength())]
        return ML.Sample(params=sample.params, spectra=spectra)
    else:
        return spectrumPreprocessor(sample)

##  Start adaptive sampling

In [ ]:
sampleAdaptively(paramRanges={'shift1': [-0.1, 0.1], 'shift2': [-0.1, 0.1]},
     spectralProgram={'generateInput':generateInput, 'parseOneFolder':parseOneFolder, 'createDataframes':createDataframes}, 
     samplePreprocessor=samplePreprocessor, workingFolder=f'sample_calc', 
     outputFolder=f'sample_result', debugFolder='sample_debug', 
     runConfiguration={'runCmd':run}, seed=0)